<a href="https://colab.research.google.com/github/stepbot/StableDiffusionNotebooks/blob/main/batchImage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

In [ ]:
!pip install diffusers[torch]==0.14.0 transformers scipy ftfy accelerate

In [ ]:
from diffusers import StableDiffusionPipeline
from diffusers import DDPMScheduler, DDIMScheduler, PNDMScheduler, EulerAncestralDiscreteScheduler
import torch
from PIL import Image
from matplotlib import pyplot as plt
from IPython import display

In [ ]:
# select model
model_id = "XpucT/Deliberate"

In [ ]:
pipe = StableDiffusionPipeline.from_pretrained(model_id,torch_dtype=torch.float16)

ddim_scheduler = DDIMScheduler.from_config(pipe.scheduler.config)
ddpm_scheduler = DDPMScheduler.from_config(pipe.scheduler.config)
pndm_scheduler = PNDMScheduler.from_config(pipe.scheduler.config)
eulerA_scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)

# set scheduler
pipe.scheduler = ddim_scheduler

pipe.safety_checker=None
pipe = pipe.to("cuda")

In [ ]:
%matplotlib inline

# set batch size
batch_size = 8

# set prompts
prompt = ""
n_prompt = "(deformed, distorted, disfigured:1.3), poorly drawn, bad anatomy, wrong anatomy, extra limb, missing limb, floating limbs, (mutated hands and fingers:1.4), disconnected limbs, mutation, mutated, ugly, disgusting, blurry, amputation. tattoo"

# generate images
images = pipe(prompt=[prompt]*batch_size, negative_prompt=[n_prompt]*batch_size, guidance_scale=8, num_inference_steps=50, height=768, width=768).images

# save images
for ii in range(len(images)):
  image = images[ii]
  image_name = "output"+str(ii)+".png"
  image.save(image_name)
  plt.figure(figsize = (8,8))
  plt.imshow(image)
  plt.axis("off")
  plt.title(image_name)
  plt.show()

